In [1]:
from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model import Quantity
from nautilus_trader.model import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.core.rust.model import TriggerType, TimeInForce
from nautilus_trader.model import TradeTick
from nautilus_trader.model import InstrumentId

In [2]:
catalog = ParquetDataCatalog("/home/p/p-dev/nautilus/catalog")
instrument_id = InstrumentId.from_str("APTUSDT-PERP.BINANCE")

In [3]:
venue = BacktestVenueConfig(
    name="BINANCE",
    oms_type="NETTING",
    account_type="MARGIN",
    base_currency="USDT",
    starting_balances=["100 USDT"],
    default_leverage=5,
    leverages={"APTUSDT-PERP.BINANCE": 5}
)

In [4]:
data = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=TradeTick,
    instrument_ids=[instrument_id],
    start_time="2024-12-10",
    end_time="2025-01-10"
)

In [5]:
engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:MACDStrategy",
            config_path="__main__:MACDConfig",
            config={
              "instrument_id": instrument_id,
              "fast_period": 12,
              "slow_period": 26,
            },
        )
    ],
    logging=LoggingConfig(log_level="INFO"),
)

In [6]:
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model import InstrumentId
from nautilus_trader.model import Position
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import PriceType
from nautilus_trader.core.rust.common import LogColor
from nautilus_trader.model.events import PositionOpened, PositionChanged, PositionClosed, OrderAccepted, OrderFilled
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.trading.strategy import StrategyConfig



class MACDConfig(StrategyConfig, frozen=True):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 10
    entry_threshold: float = 0.00010


class MACDStrategy(Strategy):
    def __init__(self, config: MACDConfig):
        super().__init__(config=config)
        self.log.info("Initialization of MACDStrategy")
        # Our "trading signal"
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period, slow_period=config.slow_period, price_type=PriceType.MID
        )

        # Convenience
        self.position: Position | None = None

    
    def on_start(self):
        self.log.info("Start of MACDStrategy")
        
        self.instrument = self.cache.instrument(self.config.instrument_id)
        self.account = self.portfolio.account(self.instrument.venue)
        self.trade_size = self.instrument.make_qty(self.config.trade_size)

        self.subscribe_trade_ticks(instrument_id=self.config.instrument_id)


    def on_stop(self):
        self.log.info("Stop of MACDStrategy")
        self.close_all_positions(self.config.instrument_id)
        self.unsubscribe_trade_ticks(instrument_id=self.config.instrument_id)

    

    def on_trade_tick(self, tick: TradeTick):
        # You can register indicators to receive quote tick updates automatically,
        # here we manually update the indicator to demonstrate the flexibility available
        self.macd.handle_trade_tick(tick)

        if not self.macd.initialized:
            return  # Wait for indicator to warm up

        # self._log.info(f"{self.macd.value=}:%5d")
        self.check_for_entry(tick)
        self.check_for_exit()


    def check_for_entry(self, tick: TradeTick):
        quantity = self.instrument.make_qty(
            self.config.trade_size
        )
        
        # If MACD line is above our entry threshold, we should be LONG
        if self.macd.value > self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.LONG:
                return  # Already LONG

            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.BUY,
                quantity=quantity,
            )
            self.position_open_price = tick.price
            self.submit_order(order)
            
        # If MACD line is below our entry threshold, we should be SHORT
        elif self.macd.value < -self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.SHORT:
                return  # Already SHORT
            
            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.SELL,
                quantity=quantity,
            )
            self.position_open_price = tick.price
            self.submit_order(order)
            

    def check_for_exit(self):
        # If MACD line is above zero then exit if we are SHORT
        if self.macd.value >= 0.0:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)

        # If MACD line is below zero then exit if we are LONG
        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)

    ''''
    def on_position_opened(self, event: PositionOpened):
        self.position = self.cache.position(event.position_id)

        if self.position.side == PositionSide.LONG:
            order_side = OrderSide.BUY

            limit_price = self.instrument.make_price(
                self.position_open_price * (1 - 0.1)
            )
            
        else:
            order_side = OrderSide.SELL

            limit_price = self.instrument.make_price(
                self.position_open_price * (1 + 0.1)
            )

        
        quantity = self.instrument.make_qty(
            self.config.trade_size
        )

        self.__limit_order = self.order_factory.limit(
            self.config.instrument_id,
            order_side,
            quantity,
            limit_price,
            reduce_only=False,
            post_only=False,
            time_in_force=TimeInForce.GTC,
            expire_time=None,
            quote_quantity=False,
            emulation_trigger=TriggerType.NO_TRIGGER,
            trigger_instrument_id=None,
            exec_algorithm_id=None,
            exec_algorithm_params=None,
            display_qty=None
        )
    '''

    
    def on_dispose(self):
        pass  # Do nothing else

In [7]:
config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data],
    chunk_size=1024*32
)

In [8]:
from nautilus_trader.backtest.results import BacktestResult


node = BacktestNode(configs=[config])

results: list[BacktestResult] = node.run()

2025-04-19T13:05:54.523630675Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-04-19T13:05:54.523658401Z [INFO] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2025-04-19T13:05:54.523660082Z [INFO] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2025-04-19T13:05:54.523660840Z [INFO] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2025. All rights reserved.
2025-04-19T13:05:54.523661712Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-04-19T13:05:54.523662715Z [INFO] BACKTESTER-001.BacktestEngine: 
2025-04-19T13:05:54.523663330Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2025-04-19T13:05:54.523664132Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2025-04-19T13:05:54.523665385Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀⠀
2025-04-19T13:05

In [9]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model import Venue


engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()

_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, account_id=BINANCE-001, opening_order_id=O-20241210-000005-001-000-1, closing_order_id=None, entry=SELL, side=SHORT, signed_qty=-350.0, quantity=350.0, peak_qty=350.0, currency=USDT, avg_px_open=11.931673657142861, avg_px_close=0.0, realized_return=0.00000, realized_pnl=-2.08804295 USDT, unrealized_pnl=0.48078000 USDT, ts_opened=1733788805048000000, ts_last=1733788806659000000, ts_closed=0, duration_ns=0)
2024-12-10T00:00:06.659000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000006-001-000-36, side=SELL, type=MARKET, quantity=10.0, time_in_force=GTC, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=None)
2024-1

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20241210-000005-001-000-1,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-001,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-002,MARKET,SELL,10.0,...,None,None,None,None,None,None,None,c6f1a457-724c-4075-aada-c19ff83fc151,2024-12-10 00:00:05.048000+00:00,2024-12-10 00:00:05.048000+00:00
O-20241210-000005-001-000-10,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-010,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-015,MARKET,SELL,10.0,...,None,None,None,None,None,None,None,a7f38f24-370b-4d20-8b0a-d7764b1117c3,2024-12-10 00:00:05.333000+00:00,2024-12-10 00:00:05.333000+00:00
O-20241210-000005-001-000-11,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-011,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-016,MARKET,SELL,10.0,...,None,None,None,None,None,None,None,451f7240-56c6-4563-ad65-3c73689534d4,2024-12-10 00:00:05.353000+00:00,2024-12-10 00:00:05.353000+00:00
O-20241210-000005-001-000-12,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-012,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-018,MARKET,SELL,10.0,...,None,None,None,None,None,None,None,c2b21cd0-196f-46b7-b3ee-3dd499fb623d,2024-12-10 00:00:05.353000+00:00,2024-12-10 00:00:05.353000+00:00
O-20241210-000005-001-000-13,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-013,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-019,MARKET,SELL,10.0,...,None,None,None,None,None,None,None,a7d54179-011f-4dff-b087-3745704b3db6,2024-12-10 00:00:05.353000+00:00,2024-12-10 00:00:05.353000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O-20241210-090806-001-000-451,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-277,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-394,MARKET,SELL,10.0,...,None,None,None,None,None,None,None,7bb6158c-517e-4c08-bf26-a97cb71a6bdf,2024-12-10 09:08:06.236000+00:00,2024-12-10 09:08:06.236000+00:00
O-20241210-100540-001-000-452,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-278,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-395,MARKET,SELL,10.0,...,None,None,None,None,None,None,None,05411630-c9ae-4875-a6a2-c3472ac2f524,2024-12-10 10:05:40.893000+00:00,2024-12-10 10:05:40.893000+00:00
O-20241210-110616-001-000-453,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-279,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-396,MARKET,SELL,10.0,...,None,None,None,None,None,None,None,87980750-7b35-478a-b1be-4114d2765651,2024-12-10 11:06:16.071000+00:00,2024-12-10 11:06:16.071000+00:00


T, last_qty=2.4, last_px=11.92870 USDT, commission=0.01431444 USDT, liquidity_side=TAKER, ts_event=1733788806863000000)
2024-12-10T00:00:06.863000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE net_position=-420.0
2024-12-10T00:00:06.863000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_maint=25.55663040 USDT
2024-12-10T00:00:06.863000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] PositionChanged(instrument_id=APTUSDT-PERP.BINANCE, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, account_id=BINANCE-001, opening_order_id=O-20241210-000005-001-000-1, closing_order_id=None, entry=SELL, side=SHORT, signed_qty=-420.0, quantity=420.0, peak_qty=420.0, currency=USDT, avg_px_open=11.931237214285717, avg_px_close=0.0, realized_return=0.00000, realized_pnl=-2.50555988 USDT, unrealized_pnl=1.06563000 USDT, ts_opened=1733788805048000000, ts_last=1733788806863000000, ts_closed=0, duration_ns=0)
2024-12-10T00:00:07.322000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[E

In [10]:
engine.trader.generate_positions_report()

ER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000007-001-000-51, side=SELL, type=MARKET, quantity=10.0, time_in_force=GTC, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=None)
2024-12-10T00:00:07.580000000Z [INFO] BACKTESTER-001.MACDStrategy: [CMD]--> SubmitOrder(order=MarketOrder(SELL 10.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-000007-001-000-51, venue_order_id=None, position_id=None, tags=None), position_id=None)
2024-12-10T00:00:07.580000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000007-001-000-51, account_id=BINANCE-001, ts_event=17337888075800

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_init,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,,
APTUSDT-PERP.BINANCE-MACDStrategy-000,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000005-001-000-1,None,SELL,SHORT,1380.0,1380.0,1733788805048000000,2024-12-10 00:00:05.048000+00:00,1733788811415000000,<NA>,None,11.923734,11.929041,[15.62311803 USDT],-0.00034,-16.15182865 USDT


USDT, maintenance=32.30290250 USDT, instrument_id=APTUSDT-PERP.BINANCE)], event_id=e67ec0bf-5b76-468a-9a49-933f99668b4d)
2024-12-10T00:00:07.596000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderFilled(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000007-001-000-54, venue_order_id=BINANCE-1-054, account_id=BINANCE-001, trade_id=BINANCE-1-083, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, order_side=SELL, order_type=MARKET, last_qty=9.1, last_px=11.92680 USDT, commission=0.05426694 USDT, liquidity_side=TAKER, ts_event=1733788807596000000)
2024-12-10T00:00:07.596000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE net_position=-540.0
2024-12-10T00:00:07.596000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_maint=32.85659700 USDT
2024-12-10T00:00:07.596000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] PositionChanged(instrument_id=APTUSDT-PERP.BINANCE, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, account_id=BINANCE-001, open

In [11]:
engine.trader.generate_account_report(Venue("BINANCE"))

VT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000007-001-000-59, side=SELL, type=MARKET, quantity=10.0, time_in_force=GTC, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=None)
2024-12-10T00:00:07.596000000Z [INFO] BACKTESTER-001.MACDStrategy: [CMD]--> SubmitOrder(order=MarketOrder(SELL 10.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-000007-001-000-59, venue_order_id=None, position_id=None, tags=None), position_id=None)
2024-12-10T00:00:07.596000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000007-001-000-59, account_id=BINANCE-001, ts_event=1733788807596000000)
2024-12-10T00:00:07

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2024-12-10 00:00:04.998000+00:00,100.00000000,0E-8,100.00000000,USDT,BINANCE-001,MARGIN,USDT,[],True,{}
2024-12-10 00:00:05.048000+00:00,99.99403215,0E-8,99.99403215,USDT,BINANCE-001,MARGIN,USDT,[],False,{}
2024-12-10 00:00:05.048000+00:00,99.94032195,0.06087207,99.87944988,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 00:00:05.048000+00:00,99.91824146,0.60871560,99.30952586,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 00:00:05.048000+00:00,99.88064526,0.83394037,99.04670489,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
...,...,...,...,...,...,...,...,...,...,...
2024-12-10 00:00:11.412000+00:00,84.02696640,81.48793260,2.53903380,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 00:00:11.414000+00:00,84.02100745,82.09536300,1.92564445,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 00:00:11.414000+00:00,83.96737735,82.15617438,1.81120297,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 00:00:11.414000+00:00,83.90777535,82.70347680,1.20429855,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}


0Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] PositionChanged(instrument_id=APTUSDT-PERP.BINANCE, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, account_id=BINANCE-001, opening_order_id=O-20241210-000005-001-000-1, closing_order_id=None, entry=SELL, side=SHORT, signed_qty=-610.0, quantity=610.0, peak_qty=610.0, currency=USDT, avg_px_open=11.92998555737705, avg_px_close=0.0, realized_return=0.00000, realized_pnl=-3.63864570 USDT, unrealized_pnl=2.73619000 USDT, ts_opened=1733788805048000000, ts_last=1733788807643000000, ts_closed=0, duration_ns=0)
2024-12-10T00:00:07.643000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000007-001-000-62, side=SELL, type=MARKET, quantity=10.0, time_in_force=GTC, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, pare